In [ ]:
!pip install transformers
!pip install pyTelegramBotAPI
!pip install wikipedia

In [1]:
import transformers
from transformers import pipeline
import telebot
import wikipedia

In [2]:
# wikipedia.summary("databases")
token = 'XXXxxx' # replace with actual token
bot = telebot.TeleBot(token, parse_mode=None)
qap = pipeline('question-answering')

In [3]:
class Responses:
    status = -1
    topic_message = "\nTell me a topic you want to know about.\n" \
                    "Topic length should be less than 300. "
    qs_message = "\nAsk a question about this topic."
    restart_message = "\nRestart ?"
    ctxt = ""
    @classmethod
    def clear_status(cls):
        cls.status = -1
        cls.ctxt = ""

In [4]:
Responses.clear_status()
@bot.message_handler(func = lambda m:True)
def reply_hi(message):
    if Responses.status == -1 and 'hi' in message.text:
        Responses.status = 0
        bot.reply_to(message, Responses.topic_message)
    elif Responses.status == 0:
        Responses.status = 1
        Responses.ctxt = wikipedia.summary(message.text)
        # print(f'context for {message.text}:\n {Responses.ctxt}')
        bot.reply_to(message, Responses.qs_message)
    elif Responses.status == 1:
        qs = message.text
        resp = qap(context = Responses.ctxt, question = qs)
        # print(f'response:\n {resp}')
        Responses.status = 0
        bot.reply_to(message, resp.get('answer', 'no answer found')+Responses.topic_message)

In [ ]:
bot.polling()